In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130510

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,147160.8167
2018-02-28,118000.3065
2018-03-31,72253.8147
2018-04-30,76373.7952
2018-05-31,46611.9031
2018-06-30,62989.8080
2018-07-31,69270.0312
2018-08-31,96517.0594
2018-09-30,93822.0026


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    147160.8167
 2018-02-28    118000.3065
 2018-03-31     72253.8147
 2018-04-30     76373.7952
 2018-05-31     46611.9031
 2018-06-30     62989.8080
 2018-07-31     69270.0312
 2018-08-31     96517.0594
 2018-09-30     93822.0026
 2018-10-31     80603.8912
 2018-11-30     81911.5318
 2018-12-31    161484.1462
 2019-01-31    187297.3466
 2019-02-28    122298.2854
 2019-03-31    175789.1879
 2019-04-30    105262.0632
 2019-05-31     58415.3425
 2019-06-30     78902.7746
 2019-07-31     75325.6490
 2019-08-31     83213.8515
 2019-09-30     69864.9708
 2019-10-31     76088.9978
 2019-11-30     91235.7225
 2019-12-31     98697.5987
 2020-01-31    152927.4603
 2020-02-29    120176.3812
 2020-03-31    213529.1218
 2020-04-30    105668.6278
 2020-05-31     92656.2156
 2020-06-30     86158.4578
 2020-07-31     97255.9319
 2020-08-31     78495.4626
 2020-09-30     87607.6489
 2020-10-31    101397.0782
 2020-11-30     76017.0460
 2020-12-31    129121.4203

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.312799
p-value : 0.000422
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1008.676, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1003.985, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1003.726, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1002.730, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1005.094, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1002.704, Time=0.18 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1001.567, Time=0.05 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1003.563, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1004.858, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1006.853, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1003.532, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1005.466, Time=0.09 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1003.492, Time=0.19 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(3,0,0)(0,0,0)

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -497.780
Method:                       css-mle   S.D. of innovations          33872.268
Date:                Sat, 11 Sep 2021   AIC                           1001.560
Time:                        04:36:34   BIC                           1006.773
Sample:                             0   HQIC                          1003.471
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.048e+05   9225.232     11.358      0.000    8.67e+04    1.23e+05
ar.L1.y        0.4438      0.138      3.214      0.001       0.173       0.714
                                    Roots           

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([ 97140.9962661 , 101387.60867213]), array([33872.2679548 , 37058.14755422]), array([[ 30752.571     , 163529.4215322 ],
       [ 28754.97413208, 174020.24321219]]))
